# Imports

In [ ]:
import pandas as pd
import numpy as np
import warnings
import random
from multiprocessing import Manager
from concurrent.futures import ProcessPoolExecutor
warnings.filterwarnings("ignore")
from PIL import Image, ImageDraw, ImageFont
import operator
import docx
from docx.shared import Pt
import os
import json

# Load data

In [ ]:
available_players = ["nsilva", "ggomes", "guest_gregs", "esousa",
                     "guest_random", "pduarte", "apimenta", "spinto",
                     "jsilva", 'gcosteira']
 
db = "players_07_23.xlsx"
season = "anygoal_season_2023_resume.csv"

# player's database
anybrain_df = pd.read_excel(db, engine='openpyxl', sheet_name='Folha4')

# Season
anygoal_df = pd.read_csv(season)
# stats
player_stats = ['Ataque', 'Tecnica', 'Tatica', 'Defesa', 'Criatividade','Fisico', 'TrabalhoEquipa', 'Ambicao', 'GK', 'MeanPoints','MeanPointsNoGR','Point_System']
# weights
weights = {'Ataque': 1, 'Tecnica': 1, 'Tatica': 1, 'Defesa': 1, 'Criatividade': 1, 'Fisico': 1, 'TrabalhoEquipa': 1, 'Ambicao': 1, 'GK': 1, 'MeanPoints': 1, 'MeanPointsNoGR': 1, 'Point_System': 0}

# Treating data 

In [ ]:
def treat_df(anybrain_df, anygoal_data):
  with open('players.json', 'r') as file:
    nomes = json.load(file)

  # Update the names
  anybrain_df['Player'] = anybrain_df['Player'].replace(nomes)

  # Merge dataframes
  merged_df = anybrain_df.merge(anygoal_data, left_on = 'Player', right_on = 'Player')

  # Removing useless stuff
  merged_df.drop(['Unnamed: 0',"Participation", "Goals", "Assists", "Own_Goals", "Result", "MVP", "BGK"], axis=1, inplace=True)

  return merged_df

In [ ]:
# Merging dataframes
merged_df = treat_df(anybrain_df, anygoal_df)
merged_df

In [ ]:
# Getting the players 
data_players = merged_df[merged_df.Player.isin(available_players)]
data_players

# Creating teams

In [ ]:
# Check the difference between teams
def evaluate_team(teamA, teamB, player_stats, weights):
    dict_A = {stat : teamA[stat].mean() * weights[stat] for stat in player_stats}
    dict_B = {stat : teamB[stat].mean() * weights[stat] for stat in player_stats}

    # Check the difference between stats  
    difference_stats = {}
    for stat in player_stats:
        difference_stats[stat] = abs(dict_A[stat] - dict_B[stat])
        
    return difference_stats

In [ ]:
# Create random teams 
def random_teams(df):
    num_jogadores = len(df)
    shuffle_index = list(range(num_jogadores))
    random.shuffle(shuffle_index)

    mid = num_jogadores // 2
    equipe1_indices = shuffle_index[:mid]
    equipe2_indices = shuffle_index[mid:]

    teamA = df.iloc[equipe1_indices].reset_index(drop=True)
    teamB = df.iloc[equipe2_indices].reset_index(drop=True)

    return teamA, teamB

In [ ]:
# Reproducing teams 
def reproduce_teams(teamA, teamB): 
        
    crossover_point = random.randint(1, len(teamA) - 1)
    
    new_teamA = pd.concat([teamA.iloc[:crossover_point], teamB.iloc[crossover_point:]]).reset_index(drop=True)
    new_teamB = pd.concat([teamB.iloc[:crossover_point], teamA.iloc[crossover_point:]]).reset_index(drop=True)
        
    return new_teamA, new_teamB

In [ ]:
# Genetic algorithm to create balanced teams 
def create_balanced_teams(df, player_stats, weights, num_gen=5000, num_sons = 20):
    # We create two random teams
    best_teamA, best_teamB = random_teams(df)
    # then we evaluate these teams  
    best_team_dict_diff = evaluate_team(best_teamA, best_teamB, player_stats, weights)
    best_team_sum_diff = sum(best_team_dict_diff.values()) / len(best_team_dict_diff)

    for geracao in range(num_gen):
        new_teamsA = []
        new_teamsB = []
        evaluate_teams = [] 
        evaluate_teams_sum_diff = []
        
        # For each generation, we will create like 20 sons
        for i in range(num_sons):
            # Reproducing the teams
            new_teamA, new_teamB = reproduce_teams(best_teamA, best_teamB)
            # Adding them to the list
            new_teamsA.append(new_teamA)
            new_teamsB.append(new_teamB)
            # Evaluate the difference between them
            diff = evaluate_team(new_teamA, new_teamB, player_stats, weights)
            evaluate_teams.append(diff)
            evaluate_teams_sum_diff.append(sum(diff.values()) /len(diff))
                    
        # Find the index where we got the lowest difference 
        indice_best_team = evaluate_teams_sum_diff.index(min(evaluate_teams_sum_diff))        

        # Compare that one with the best result until now
        if evaluate_teams_sum_diff[indice_best_team] < best_team_sum_diff:
            best_teamA = new_teamsA[indice_best_team]
            best_teamB = new_teamsB[indice_best_team]
            best_team_dict_diff = evaluate_teams[indice_best_team]
            best_team_sum_diff = evaluate_teams_sum_diff[indice_best_team]
            
    
    return best_teamA, best_teamB, best_team_dict_diff, best_team_sum_diff
    

In [ ]:
equipaA, equipaB, stats, diff = create_balanced_teams(data_players, player_stats, weights)

In [ ]:
equipaA, equipaB

In [ ]:
diff

# Creating images 

In [ ]:
def build_game_sheet(A, B):
    # Prepare the initial settings
    tmp = ["Jogador_1", "Jogador_2", "Jogador_3", "Jogador_4", "Jogador_5", "Jogador_6", "Jogador_7", "Jogador_8", "Jogador_9", "Jogador_10"]
    tmp2 = A + B
    full_players = dict(zip(tmp,tmp2))
    
    # Remove any previous existing game sheet
    if os.path.exists("./Anygoal_Template.docx"):
        os.remove("./Anygoal_Template.docx")

    # Open the template file
    doc = docx.Document("./template/Anygoal_Template.docx")
    
    # Search any cell with string equal to tmp and replace it with tmp2 values
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if cell.text in full_players:
                    cell.text = cell.text.replace(cell.text, full_players[cell.text])

    # Save the new game sheet
    doc.save("./Anygoal_Template.docx")

# Deciding whos washing


In [ ]:
def whos_washing(df):
    # Open and create the image object so it can be ready to be edited
    
    winner = df.sample(n=1)['Player'].values[0]    
    
    starter = Image.open("./template/who_washing.png")
    draw = ImageDraw.Draw(starter) 
    # Font
    font = ImageFont.truetype("./template/newsgott-regular.ttf", 50)
    options = {'bold':True}

    draw.text((270, 90), winner.upper(), font=font, **options)    
    starter.save("winner.png")

In [ ]:
whos_washing(data_players)